# Cats vs Non-Cats Classification (Deep Neural Network)

Colab-ready notebook using NumPy and H5 datasets.

## Upload dataset files
Upload `train_catvnoncat.h5` and `test_catvnoncat.h5` to the Colab environment.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import h5py


## Load Dataset

In [ ]:

def load_dataset():
    train_dataset = h5py.File('train_catvnoncat.h5', "r")
    test_dataset = h5py.File('test_catvnoncat.h5', "r")

    X_train = np.array(train_dataset["train_set_x"][:])
    Y_train = np.array(train_dataset["train_set_y"][:])
    X_test = np.array(test_dataset["test_set_x"][:])
    Y_test = np.array(test_dataset["test_set_y"][:])

    Y_train = Y_train.reshape(1, -1)
    Y_test = Y_test.reshape(1, -1)

    return X_train, Y_train, X_test, Y_test


## Preprocess Data

In [ ]:

X_train, Y_train, X_test, Y_test = load_dataset()

X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(X_train.shape[0], -1).T
X_test = X_test.reshape(X_test.shape[0], -1).T

print("Train:", X_train.shape)
print("Test:", X_test.shape)


## Neural Network Functions

In [ ]:

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def relu(Z):
    return np.maximum(0, Z)

def initialize_parameters(n_x, n_h, n_y):
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    return {"W1":W1,"b1":b1,"W2":W2,"b2":b2}


In [ ]:

def forward_propagation(X, params):
    Z1 = np.dot(params["W1"], X) + params["b1"]
    A1 = relu(Z1)
    Z2 = np.dot(params["W2"], A1) + params["b2"]
    A2 = sigmoid(Z2)
    return A2, {"A1":A1,"A2":A2}


In [ ]:

def compute_loss(A2, Y):
    m = Y.shape[1]
    return -(1/m) * np.sum(Y*np.log(A2) + (1-Y)*np.log(1-A2))


In [ ]:

def backward_propagation(X, Y, params, cache):
    m = X.shape[1]
    dZ2 = cache["A2"] - Y
    dW2 = (1/m) * np.dot(dZ2, cache["A1"].T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = np.dot(params["W2"].T, dZ2)
    dZ1[cache["A1"] <= 0] = 0
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

    return {"dW1":dW1,"db1":db1,"dW2":dW2,"db2":db2}


In [ ]:

def update_parameters(params, grads, lr):
    for key in params:
        params[key] -= lr * grads["d"+key]
    return params


## Train Model

In [ ]:

def train_model(X, Y, n_h=64, epochs=2000, lr=0.01):
    params = initialize_parameters(X.shape[0], n_h, 1)
    losses = []
    for i in range(epochs):
        A2, cache = forward_propagation(X, params)
        loss = compute_loss(A2, Y)
        grads = backward_propagation(X, Y, params, cache)
        params = update_parameters(params, grads, lr)
        if i % 100 == 0:
            losses.append(loss)
    return params, losses


In [ ]:

params, losses = train_model(X_train, Y_train)


## Evaluate & Plot

In [ ]:

def predict(X, params):
    A2,_ = forward_propagation(X, params)
    return (A2 > 0.5)

train_acc = np.mean(predict(X_train, params) == Y_train) * 100
test_acc = np.mean(predict(X_test, params) == Y_test) * 100

print("Training Accuracy:", train_acc)
print("Testing Accuracy:", test_acc)

plt.plot(losses)
plt.xlabel("Epochs (x100)")
plt.ylabel("Loss")
plt.title("Loss vs Epochs")
plt.show()
